In [1]:
import logging
import json
import torch
from types import SimpleNamespace
from vilbert.vilbert import VILBertActionGrounding, BertConfig
from pytorch_transformers.tokenization_bert import BertTokenizer
from pytorch_transformers.optimization import AdamW, WarmupLinearSchedule
import torch.distributed as dist
from VLN_config import config as args
import random
import pandas as pd
from data.dataLoader import DataLoader

In [2]:
import sys
import os
import torch
import yaml

import numpy as np
import matplotlib.pyplot as plt
import PIL

from PIL import Image
import cv2
import argparse
import glob
import pdb

import torchvision.models as models
import torchvision.transforms as transforms

from faster_rcnn import feature_extractor_new as f_extractor
from faster_rcnn.feature_extractor_new import featureExtractor
#%matplotlib inline  

In [3]:
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger = logging.getLogger(__name__)
print(args)

namespace(adam_epsilon=1e-08, baseline=False, bert_model='bert-base-uncased', best_features=5, clean_train_sets=True, clip_size=3, config_file='config/bert_base_6layer_6conect.json', distributed=False, do_lower_case=True, dynamic_attention=False, epochs=1000, from_pretrained='save/multitask_model/multi_task_model.bin', gradient_accumulation_steps=1, img_weight=1, in_memory=False, learning_rate=0.0001, local_rank=-1, max_temporal_memory_buffer=3, mean_layer=False, num_key_frames=2, num_train_epochs=10.0, num_workers=0, objective=1, predict_feature=False, save_name='', seed=42, split='mteval', start_epoch=0, task_specific_tokens=True, tasks='1', threshold_similarity=0.7, track_temporal_features=True, train_batch_size=4, use_tensorboard=True, visual_target=0, warmup_proportion=0.1, without_coattention=False)


In [4]:
tokenizer = BertTokenizer.from_pretrained(
    args.bert_model, do_lower_case=args.do_lower_case
)

07/31/2020 15:36:19 - INFO - pytorch_transformers.tokenization_utils -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/drissh/.cache/torch/pytorch_transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [5]:
config = BertConfig.from_json_file(args.config_file)
bert_weight_name = json.load(
    open("config/" + args.bert_model + "_weight_name.json", "r")
)

tokenizer = BertTokenizer.from_pretrained(
    args.bert_model, do_lower_case=args.do_lower_case
)

config.track_temporal_features = args.track_temporal_features
config.mean_layer = args.mean_layer
config.max_temporal_memory_buffer = args.max_temporal_memory_buffer
config.visualization = True

#The path of the finetuned ActionGrounding weights
# args.from_pretrained = "save_vilbert_action_grounding/best_val_vilberActionGrounding.bin"

print(args.from_pretrained)
model = VILBertActionGrounding.from_pretrained(
    args.from_pretrained, config=config, default_gpu=True
)
model.eval()

07/31/2020 15:36:20 - INFO - pytorch_transformers.tokenization_utils -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/drissh/.cache/torch/pytorch_transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
07/31/2020 15:36:20 - INFO - vilbert.utils -   loading weights file save/multitask_model/multi_task_model.bin


save/multitask_model/multi_task_model.bin


07/31/2020 15:36:29 - INFO - vilbert.utils -   Weights of VILBertActionGrounding not initialized from pretrained model: ['positional_enc.weight', 'positional_enc.bias', 'img_emb_mean.weight', 'img_emb_mean.bias', 'action_cls.predictions.bias', 'action_cls.predictions.transform.dense.weight', 'action_cls.predictions.transform.dense.bias', 'action_cls.predictions.transform.LayerNorm.weight', 'action_cls.predictions.transform.LayerNorm.bias', 'action_cls.predictions.decoder.weight', 'action_cls.bi_seq_relationship.weight', 'action_cls.bi_seq_relationship.bias', 'action_cls.imagePredictions.transform.dense.weight', 'action_cls.imagePredictions.transform.dense.bias', 'action_cls.imagePredictions.transform.LayerNorm.weight', 'action_cls.imagePredictions.transform.LayerNorm.bias', 'action_cls.imagePredictions.decoder.weight', 'action_cls.imagePredictions.decoder.bias']
07/31/2020 15:36:29 - INFO - vilbert.utils -   Weights from pretrained model not used in VILBertActionGrounding: ['cls.predic

VILBertActionGrounding(
  (positional_enc): Linear(in_features=7, out_features=2048, bias=True)
  (img_emb_mean): Linear(in_features=6144, out_features=2048, bias=True)
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (v_embeddings): BertImageEmbeddings(
      (image_embeddings): Linear(in_features=2048, out_features=1024, bias=True)
      (image_location_embeddings): Linear(in_features=5, out_features=1024, bias=True)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)

In [6]:
# load data
frcnn_model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
data_loader = DataLoader("data/json_data.json", "data/json_token_count.json", frcnn_model, save_or_not = False)
path = 'data/DataLoader.pt'
data_loaded = data_loader.load_dataloader(path)

18


In [7]:
features_masked, pos_enc, spatial, image_mask, tokenized_text, masked_text, masked_lm_token, input_mask, segment_ids, co_attention_mask, infos, masked_img_labels  = data_loaded

In [33]:
infos[0][0]

{'bbox': array([[ 67.43359 ,  33.40821 , 288.82733 ,  85.165436],
        [ 89.2899  ,  35.211536, 175.78989 ,  78.11629 ],
        [185.62347 ,  36.098824, 262.3924  ,  71.7224  ],
        [187.37694 ,  35.066093, 250.65431 ,  58.121117],
        [168.40512 ,  32.706028, 286.94073 ,  80.84946 ],
        [182.13742 ,  98.88083 , 292.50766 , 177.57565 ],
        [ 13.630596,  79.63491 ,  43.086037, 109.07181 ],
        [267.72318 ,  41.43473 , 286.8675  ,  60.650494],
        [260.55362 ,  79.71673 , 300.      , 132.79883 ],
        [  0.323164,  76.30916 , 149.61972 , 199.20862 ]], dtype=float32),
 'pos_enc': array([[ 67.43359 ,  33.40821 , 288.82733 ,  85.165436,   0.      ,
          17.      ,  16.      ],
        [ 89.2899  ,  35.211536, 175.78989 ,  78.11629 ,   1.      ,
          17.      ,  16.      ],
        [185.62347 ,  36.098824, 262.3924  ,  71.7224  ,   2.      ,
          17.      ,  16.      ],
        [187.37694 ,  35.066093, 250.65431 ,  58.121117,   3.      ,
      

In [8]:
#Forward the model with one input example
model.eval()
pred_t_train, pred_v_train, att_train = model(input_ids = masked_text.cpu(),
                                image_feat = features_masked.cpu(), # Linear(2048*config.max_temporal_memory_buffer, 2048)
                                image_loc = spatial.cpu(),  #Linear(in_features=5, out_features=1024, bias=True)
                                image_pos_input = pos_enc.cpu(),   #Linear(7, 2048)/(6, 2048)
                                token_type_ids = segment_ids.cpu(), 
                                attention_mask = input_mask.cpu(), 
                                image_attention_mask = image_mask.cpu(),
                                output_all_attention_masks=True)

## Text prediction and attention

In [9]:
i=3
print(tokenizer.convert_ids_to_tokens(masked_text[i].tolist()))
#To set which word is masked
for j in range(len(masked_lm_token[i])):
    if masked_lm_token[i][j].item()!=-1:
        break
id_word_masked = torch.argmax(pred_t_train[0,j,:]).item()
print("predicted token" , id_word_masked)
print("gt token : ", masked_lm_token[i][j].item())
predicted_masked_word = tokenizer.convert_ids_to_tokens(id_word_masked)
print("Prediction: ", predicted_masked_word)
print("GT: ", tokenizer.convert_ids_to_tokens(masked_lm_token[i][j].item()))

['[CLS]', 'wash', '[MASK]', 'lad', '##le', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
predicted token 14412
gt token :  1996
Prediction:  pal
GT:  the


In [10]:
%%javascript
require.config({
  paths: {
      d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min',
      jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
  }
});

<IPython.core.display.Javascript object>

In [11]:
import torch

def format_attention(attention):
    squeezed = []
    for layer_attention in attention:
        # 1 x num_heads x seq_len x seq_len
        if len(layer_attention.shape) != 4:
            raise ValueError("The attention tensor does not have the correct number of dimensions. Make sure you set "
                             "output_attentions=True when initializing your model.")
        squeezed.append(layer_attention.squeeze(0))
    # num_layers x num_heads x seq_len x seq_len
    return torch.stack(squeezed)

def format_special_chars(tokens):
    return [t.replace('Ġ', ' ').replace('▁', ' ').replace('</w>', '') for t in tokens]

In [12]:
import json
from IPython.core.display import display, HTML, Javascript
import os



def head_view(attention, tokens, tokens_l, sentence_b_start = None, prettify_tokens=False):
    """Render head view
        Args:
            attention: list of ``torch.FloatTensor``(one for each layer) of shape
                ``(batch_size(must be 1), num_heads, sequence_length, sequence_length)``
            tokens: list of tokens
            sentence_b_index: index of first wordpiece in sentence B if input text is sentence pair (optional)
            prettify_tokens: indicates whether to remove special characters in wordpieces, e.g. Ġ
    """

    if sentence_b_start is not None:
        vis_html = """
        <span style="user-select:none">
            Layer: <select id="layer"></select>
            Attention: <select id="filter">
              <option value="all">All</option>
              <option value="aa">Sentence A -> Sentence A</option>
              <option value="ab">Sentence A -> Sentence B</option>
              <option value="ba">Sentence B -> Sentence A</option>
              <option value="bb">Sentence B -> Sentence B</option>
            </select>
            </span>
        <div id='vis'></div>
        """
    else:
        vis_html = """
              <span style="user-select:none">
                Layer: <select id="layer"></select>
              </span>
              <div id='vis'></div> 
            """

    display(HTML(vis_html))
    __location__ = os.path.realpath(
        os.path.join(os.getcwd()))#, os.path.dirname(__file__)))
    vis_js = open(os.path.join(__location__, 'head_view.js')).read()

    if prettify_tokens:
        tokens = format_special_chars(tokens)

    attn = format_attention(attention)
    attn_data = {
        'all': {
            'attn': attn.tolist(),
            'left_text': tokens,
            'right_text': tokens_l
        }
    }
    if sentence_b_start is not None:
        slice_a = slice(0, sentence_b_start)  # Positions corresponding to sentence A in input
        slice_b = slice(sentence_b_start, len(tokens))  # Position corresponding to sentence B in input
        attn_data['aa'] = {
            'attn': attn[:, :, slice_a, slice_a].tolist(),
            'left_text': tokens[slice_a],
            'right_text': tokens[slice_a]
        }
        attn_data['bb'] = {
            'attn': attn[:, :, slice_b, slice_b].tolist(),
            'left_text': tokens[slice_b],
            'right_text': tokens[slice_b]
        }
        attn_data['ab'] = {
            'attn': attn[:, :, slice_a, slice_b].tolist(),
            'left_text': tokens[slice_a],
            'right_text': tokens[slice_b]
        }
        attn_data['ba'] = {
            'attn': attn[:, :, slice_b, slice_a].tolist(),
            'left_text': tokens[slice_b],
            'right_text': tokens[slice_a]
        }
    params = {
        'attention': attn_data,
        'default_filter': "all"
    }
    attn_seq_len = len(attn_data['all']['attn'][0][0])
    print(attn_seq_len)
    if attn_seq_len != len(tokens):
        pass
        #raise ValueError(f"Attention has {attn_seq_len} positions, while number of tokens is {len(tokens)}")

    display(Javascript('window.params = %s' % json.dumps(params)))
    display(Javascript(vis_js))

In [37]:
# (all_attention_mask_t, all_attnetion_mask_v, all_attention_mask_c) = att_train
def preprocess_att_viz(data_idx, att_idx):
    global att_train, infos
    print(att_train[att_idx][0].keys())
    att_list = []
    for i in range(len(att_train[att_idx])):
        if att_idx == 2:
            att_list.append(att_train[att_idx][i]["attn1"][data_idx].unsqueeze(0))
        else:
            att_list.append(att_train[att_idx][i]["attn"][data_idx].unsqueeze(0))
    input_id_list = masked_text[data_idx].tolist()
    tokens = tokenizer.convert_ids_to_tokens(input_id_list) 
#     img_reg = [1]*att_list[att_idx].shape[2] + [0]*att_list[att_idx].shape[-1]
    img_reg = infos[data_idx][0]["objects"].tolist() # + ['PAD']*abs(att_list[att_idx].shape[-1] - infos[data_idx]["objects"].shape[0])  
    if att_idx==0:
        return att_list, tokens, tokens
    if att_idx==1:
        return att_list, img_reg, img_reg
    else:
        return att_list, tokens, img_reg

In [39]:
att_list, token1, token2= preprocess_att_viz(0, 0)
head_view(att_list, token2, token1)

dict_keys(['attn', 'queries', 'keys'])


37


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [43]:
att_list[0][0, 0, :, :]

tensor([[0.5636, 0.0197, 0.0466,  ..., 0.0000, 0.0000, 0.0000],
        [0.1129, 0.2092, 0.1177,  ..., 0.0000, 0.0000, 0.0000],
        [0.5786, 0.0464, 0.0546,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.5742, 0.0366, 0.0272,  ..., 0.0000, 0.0000, 0.0000],
        [0.5631, 0.0359, 0.0274,  ..., 0.0000, 0.0000, 0.0000],
        [0.5979, 0.0331, 0.0303,  ..., 0.0000, 0.0000, 0.0000]],
       grad_fn=<SliceBackward>)

## Image prediction and attention

In [12]:
i=2
#print(tokenizer.convert_ids_to_tokens(masked_text[i].tolist()))
#To set which word is masked
print(masked_img_labels)
for j in range(len(masked_img_labels[i])):
    if masked_img_labels[i][j].item()!=-1:
        break
print(masked_img_labels[i])
id_word_masked = torch.argmax(pred_v_train[i,j,:]).item()
print("predicted token" , id_word_masked)
print("gt token : ", masked_img_labels[i][j].item())
predicted_masked_word = tokenizer.convert_ids_to_tokens(id_word_masked)
print("Prediction: ", predicted_masked_word)
print("GT: ", tokenizer.convert_ids_to_tokens(masked_img_labels[i][j].item()))

tensor([[-1, 81, -1, -1, -1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, 62, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, 31, -1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1, 81, -1, -1, -1],
        [-1, -1, 85, -1, -1, -1, -1, -1, -1, -1, -1],
        [-1, 81, -1, -1, -1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, 70, -1, -1, -1, -1, -1],
        [-1, -1, -1, 81, -1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, 53, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, 53, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1, 78, -1, -1, -1],
        [-1, -1, -1, -1, -1, 78, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, 81, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1, -1, -1, 70, -1]])
tensor([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1])
predicted token 66
gt token :  -1
Prediction:  [unused65]
GT:  [UNK]
